In [1]:
import pandas as pd
import numpy as np

from matplotlib import pylab as plt
import networkx as nx

Analysing the graph of hierarcy

In [53]:
#visualize graph
def save_graph(graph,file_name):
    #initialze Figure
    plt.figure(num=None, figsize=(20, 20), dpi=80)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos)

    cut = 1.00
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(0, xmax)
    plt.ylim(0, ymax)

    plt.savefig(file_name,bbox_inches="tight")
    plt.close()
    del fig

This exercise is to remove a part of hierarchy to make data managable for my system

In [4]:
#create directed graph from hierarchy
def create_digraph(file):
    graph = nx.DiGraph()

    file = open(file)

    lines = file.readlines()

    for line in lines:
        edge = line.rstrip().split(' ')
        graph.add_edge(int(edge[0]), int(edge[1]))
    file.close()
    return graph

graph = create_digraph('hierarchy.txt')
all_nodes = nx.nodes(graph)


In [5]:
print(len(all_nodes))
nodes_to_remove = all_nodes[0:250000]
print(len(nodes_to_remove))
graph.remove_nodes_from(nodes_to_remove)
print(graph.number_of_nodes())

478020
250000
228020


After removing some nodes from the graph we now look for largest connected component.

In [6]:
#find a connected component of graph with maximum number of nodes
def single_connected_graph(graph):
    new_graphs = nx.weakly_connected_component_subgraphs(graph)

    the_graphs = []

    for g in new_graphs:
        the_graphs.append(g)
    max_nodes = 0
    max_nodes_index = 0
    index = 0
    for g in the_graphs:
        num_nodes = g.number_of_nodes()
        if(num_nodes > max_nodes):
            max_nodes = num_nodes
            max_nodes_index = index
        index += 1
    final_graph = the_graphs[max_nodes_index]
    return final_graph

final_graph = single_connected_graph(graph)
final_nodes = nx.nodes(final_graph)
print(len(final_nodes))
# nx.write_edgelist(final_graph, "new_hierarchy.txt")

147739


These are the nodes in a subset of heirarchy

In [7]:
discarded_nodes = []
map_final_nodes = {}

for node in final_nodes:
    map_final_nodes[node] = 1

for node in all_nodes:
    if(node not in map_final_nodes):
        discarded_nodes.append(node)
    
map_discarded_nodes = {}

for node in discarded_nodes:
    map_discarded_nodes[node] = 1
print(len(discarded_nodes))
print(discarded_nodes[0:10])

330281
[1048576, 1048577, 2, 174763, 4, 2097157, 6, 1, 8, 2097161]


Now we filter our data to retain only the data with nodes in our retained hierarchy.

In [8]:
data = open('train-remapped.csv', 'r')
output_file = open('final_training.csv', 'w')
lines = data.readlines()
output_file.write(lines[0])
for i in range(1,len(lines)):
#     print(lines[i])
    classes = lines[i].rstrip().split(":")[0]
    true_classes = []
    for j in range(len(classes)):
        index = len(classes) -j - 1
        if(classes[index] == ' '):
            true_classes = classes[0:index].split(',')
            break
    flag = False
    for c in true_classes:
        if int(c) in map_discarded_nodes:
            flag = True
            break
    if(not flag):
        output_file.write(lines[i])
    
    
data.close()
output_file.close()

In [9]:
print(len(lines))

output = open('final_training.csv', 'r')
print(len(output.readlines()))

2365437
216313


In [10]:
nx.is_directed_acyclic_graph(final_graph)

False

It turns out that even the reduced hierarchy is not a DAG. This is problamatic for any meaningful traversal through the heirarchy.

Another way of looking at the data would be looking at most recurring classes and those should be the classes we may focus on first.

In [47]:
#count number of examples for each class
import operator
def count_concepts(file_name):
    file = open(file_name, 'r')
    lines = file.readlines()
    class_map ={}
    for i in range(1, len(lines)):
        classes = lines[i].rstrip().split(":")[0]
        true_classes = []
        for j in range(len(classes)):
            index = len(classes) -j - 1
            if(classes[index] == ' '):
                true_classes = classes[0:index].split(',')
                break
        for c in true_classes:
            if int(c) in class_map:
                class_map[int(c)] = class_map[int(c)] + 1
            else:
                class_map[int(c)] = 1
    file.close()
    return class_map

In [59]:
class_map = count_concepts('train-remapped.csv')

sorted_list = sorted(class_map.items(), key=operator.itemgetter(1), reverse=True)

print(sorted_list[0:10])
print(sorted_list[-10:-1])
print(len(filter(lambda x: x[1] == 1, sorted_list)))

[(24177, 387168), (285613, 41104), (98808, 14838), (264962, 12556), (167593, 11400), (242532, 10435), (52954, 10026), (300558, 9473), (444502, 9217), (78249, 9161)]
[(445683, 1), (445690, 1), (445691, 1), (445699, 1), (445707, 1), (445710, 1), (445717, 1), (445722, 1), (445724, 1)]
43368


Some classes are dispropotionately popular while a lot (43368) have only 1 sample available. So we look for classes which have got some reasonable number of sample available.

In [60]:
#Find all classes with atleast 1000 examples in dataset
relevant_classes = list(map(lambda x: x[0], list(filter(lambda x: x[1] >= 10000, sorted_list))))
print(relevant_classes)
print(len(relevant_classes))

relevant_classes_map = {}

for c in relevant_classes:
    relevant_classes_map[c] = 1
# print(relevant_classes_map.keys())

[24177, 285613, 98808, 264962, 167593, 242532, 52954]
7


If we consider classes which have atleast 10000 samples then we get onl 7 classes in case of atleast 1000 samples likewise there are around 350 classes.

In [80]:
hfile = open('hierarchy.txt', 'r')
newhfile = open('updated_hierarchy.txt', 'w')
for line in hfile.readlines():
    x = line.rstrip().split(' ')
    
    if(int(x[0]) in relevant_classes_map or int(x[1]) in relevant_classes_map):
        newhfile.write(line)
hfile.close()
newhfile.close()
        

In [4]:
# train_file = open('train-remapped.csv', 'r')
# update_train_file = open('train-updated.csv', 'w')
# lines = train_file.readlines()
# # update_train_file.write(lines[0])
# for i in range(1, len(lines)):
#     classes = lines[i].rstrip().split(":")[0]
#     true_classes = []
#     for j in range(len(classes)):
#         index = len(classes) -j - 1
#         if(classes[index] == ' '):
#             true_classes = classes[0:index].split(',')
#             break
# #     for c in true_classes:
# #         if int(c) in relevant_classes_map:
#     new_classes = get_new_classes(true_classese)
#     if(len(true_classes) > 0):
#         update_train_file.write(lines[i])
# #     break
# train_file.close()
# update_train_file.close()

In [24]:
update_train_file = open('train-updated.csv', 'r')
print(len(update_train_file.readlines()))

723085


In [50]:
def get_new_classes(classes):
    new_classes = []
    for c in classes:
        if c in relevant_classes_map:
            new_classes.append(c)
    return list(map(str, new_classes))

In [11]:
# train_file = open('train-remapped.csv', 'r')
# update_train_file = open('train-updated-small.csv', 'w')
# lines = train_file.readlines()
# # update_train_file.write(lines[0])
# for i in range(1, len(lines)):
#     classes = lines[i].rstrip().split(":")[0]
#     features = ''
#     true_classes = []
#     for j in range(len(classes)):
#         index = len(classes) -j - 1
#         if(classes[index] == ' '):
#             true_classes = list(map(int, classes[0:index].split(',')))
#             features = lines[i].split(classes[0:index])[1]
#             break
# #     for c in true_classes:
# #         if int(c) in relevant_classes_map:

#     new_classes = get_new_classes(true_classes)
# #     print(new_classes)
# #     print(features)
#     if(len(new_classes) > 0):
#         update_train_file.write(str(",".join(new_classes)) + features)
# #     break
# train_file.close()
# update_train_file.close()

#This method is for measuring recall and  mistake(when a sample is assigned only one class and it is not among any valid class for the sample. 

In [105]:
def multilabel_performance(pred, true, indexes):
    
    spliced_indexes  = []
    previous_index = indexes[0]
    temp_indexes = []
    total_score = 0
    total_mistake = 0
    for i in range(len(indexes)):
        if(indexes[i] != previous_index):
            spliced_indexes.append(temp_indexes)
            temp_indexes = []
        temp_indexes.append(i)
    spliced_indexes.append(temp_indexes)
    total_samples = len(spliced_indexes)
    for same_index  in spliced_indexes:
        temp_score = 0
        for x in same_index:
            if(pred[x] == true[x]):
                temp_score = 1
        if(temp_score == 1):
            total_score += 1
            temp_score = 0
        else:
            total_mistake += 1
    return float(total_score)/total_samples, float(total_mistake)/total_samples
            
        
        
        

#Here we convert the data from multilabel to single label for easy manipulation. We also retain their indexes in case a sample falls in more than one class. These indexes will be required while measuring accuracy of prediction. When a sample has multiple labels we create a separate row for it with each label.

In [94]:
train_file = open('train-remapped.csv', 'r')
update_train_file = open('train-updated-single-label-small.csv', 'w')
index_file = open('index.csv', 'w')
lines = train_file.readlines()
# update_train_file.write(lines[0])
k = 0
for i in range(1, len(lines)):
    classes = lines[i].rstrip().split(":")[0]
    features = ''
    true_classes = []
    for j in range(len(classes)):
        index = len(classes) -j - 1
        if(classes[index] == ' '):
            true_classes = list(map(int, classes[0:index].split(',')))
            features = lines[i].split(classes[0:index])[1]
            break
#     for c in true_classes:
#         if int(c) in relevant_classes_map:

    new_classes = get_new_classes(true_classes)
#     print(new_classes)
#     print(features)
    flag = False
    for c in new_classes:
        update_train_file.write(c + features)
        index_file.write(str(k) + '\n')
        flag = True
    if(flag):
        k += 1
        flag = False
        
#     break
train_file.close()
update_train_file.close()
index_file.close()

In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.datasets import load_svmlight_file
import sklearn
import mdp

In [96]:
dataf = load_svmlight_file("train-updated-single-label-small.csv")
indexes = []

with open('index.csv', 'r') as f:
    for x in f.readlines():
        indexes.append(int(x))

In [97]:
train_samples = 100000
test_samples = 100000
X_train = dataf[0][0:train_samples]
y_train = dataf[1][0:train_samples]
indexes_train = indexes[0:train_samples]
X_test = dataf[0][train_samples:train_samples + test_samples]
y_test = dataf[1][train_samples:train_samples + test_samples]
indexes_test = indexes[train_samples:train_samples + test_samples]
print(len(indexes))

487527


#Testing SVM classification on a small sample and see how it performs. 

In [11]:
model = sklearn.svm.SVC(verbose=3)
model.fit(X_train,y_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=3)

In [106]:
train_pred = model.predict(X_train)


print(sklearn.metrics.confusion_matrix(train_pred, y_train))
print(sklearn.metrics.accuracy_score(train_pred, y_train))
print(multilabel_performance(train_pred, y_train, indexes_train))

print("*" * 100)
test_pred = model.predict(X_test)

print(sklearn.metrics.confusion_matrix(test_pred, y_test))
print(sklearn.metrics.accuracy_score(test_pred, y_test))
print(multilabel_performance(test_pred, y_test, indexes_test))

[[76790  1000  3739  1910  4811  5488  6262]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]]
0.7679
(0.7679, 0.2321)
****************************************************************************************************
[[78695  1904  2777  1864  2318  2634  9808]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0]]
0.78695
(0.78695, 0.21305)


#Using svm to train a model. The model is very simple it predicts every sample as the most popular class. In doing so it may reach around .787 recall. To do better lets use a simple decision tree.

In [43]:
modeldecisionTree = sklearn.tree.DecisionTreeClassifier()
modeldecisionTree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [107]:
train_pred = modeldecisionTree.predict(X_train)
print(sklearn.metrics.confusion_matrix(train_pred, y_train))
print(sklearn.metrics.accuracy_score(train_pred, y_train))
print(multilabel_performance(train_pred, y_train, indexes_train))


print("*" * 100)
test_pred = modeldecisionTree.predict(X_test)
print(sklearn.metrics.confusion_matrix(test_pred, y_test))
print(sklearn.metrics.accuracy_score(test_pred, y_test))
print(multilabel_performance(test_pred, y_test, indexes_test))

[[76790   800     0     0  3880  3882  6261]
 [    0   200     0     0     0     0     0]
 [    0     0  3739  1425     0     0     0]
 [    0     0     0   485     0     0     0]
 [    0     0     0     0   931   835     0]
 [    0     0     0     0     0   771     0]
 [    0     0     0     0     0     0     1]]
0.82917
(0.82917, 0.17083)
****************************************************************************************************
[[77879  1717   339   224  2013  2037  9542]
 [   53   186     0     0     0     0     3]
 [  525     0  2266  1466    16    51   202]
 [   31     0   160   169     2     5     7]
 [  118     0     7     3   226   288    39]
 [   87     1     5     2    61   253    15]
 [    2     0     0     0     0     0     0]]
0.80979
(0.80979, 0.19021)


###With decision tree there is improvement. Shifting of prediction towards diagonal is observed. When base line is that good much improvement is difficult. There is not much overfitting too as train and test performance is close